# AI FAQ 챗봇 만들기 with RAG

In [5]:
# 환경변수 설정
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
import json

1. 데이터 로드

In [7]:
with open('./data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    faq_data = json.load(f)

In [8]:
faq_data

[{'question': '반품 정책이 어떻게 되나요?',
  'answer': '제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'},
 {'question': '배송은 얼마나 걸리나요?',
  'answer': '일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'},
 {'question': '회원가입 없이 주문할 수 있나요?',
  'answer': '네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'},
 {'question': '포인트 적립은 어떻게 되나요?',
  'answer': '구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'},
 {'question': '해외 배송이 가능한가요?',
  'answer': '일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'},
 {'question': '제품 보증 기간은 얼마나 되나요?',
  'answer': '제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'},
 {'question': 'A/S 신청은 어떻게 하나요?',
  'answer': '공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'},
 {'question': '재고가 없는 상품은 언제 다시 입고되나요?',
  'answer': '입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'},
 {'question': '구매 후 영수증을 받을 수 있나요?',
  'answer': '네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'},
 {'question': '다른 사람이 대신 주문을 수령할 수 있나요?',
  'answer': '네, 수령인을 지정할 수 있으며 신분증 확인 후 수령 가능합니다.'},
 {'question': '결제 방법에는 어떤 것이 있나요?',
  'answer': '신용카드, 

2. embedding 모델 생성 + vector store 생성

In [ ]:
embedding_model = OpenAIEmbeddings()
# print(embedding_model.model)

vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='./chroma_db'
)

documents = [Document(page_content=item['question']+" "+item['answer']) for item in faq_data]
vector_store.add_documents(documents)

3. llm 설정 (모델 생성)

In [ ]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

4. 프롬프트 및 체인 생성

In [ ]:
prompt = PromptTemplate(
    template="""
다음 '질문'에 대해 '문서'의 내용만을 참고하여 '답변'을 생성해 주세요.
질문 : {question} 
문서 : {context}
답변 :
""",
    input_variables=['question', 'context']
)

# prompt를 구성할 때 retriever의 검색 결과로 들어갈 input_variable은 'context'로 설정해야 함

In [ ]:
retriever = vector_store.as_retriever()
retriever

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

5. 사용

In [ ]:
qa_chain.run('배송 얼마나 걸려?')

In [ ]:
qa_chain.run('구매 취소하고 싶은데')

In [ ]:
qa_chain.run('구매 영수증을 발급해줘')